In [1]:
#import the necessary packages
import sys
assert sys.version_info >= (3, 5)

#Import keras, sklearn, tensorflow
import tensorflow as tf
from tensorflow import keras
import sklearn

%load_ext tensorboard


#import other common packages
import numpy as np
import os
#!pip install tqdm ipywidgets

In [2]:
#divide the training data to further training and validation set
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test.astype(np.float32) / 255.

## Create the custom loop

#### Start by creating a simple model

In [3]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation = "softmax"),
])

### Create a function that randomly selects a batch from the mnist dataset

In [4]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size= batch_size)
    return X[idx], y[idx]

### Define a function that will display the training status, including number of steps

In [5]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end =  "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)

### Now let's create the custom looping function

In [ ]:
#start by defining some hyperparameters
n_epochs = 5
batch_size=32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.sparse_categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.SparseCategoricalAccuracy()]

#import trange
from tqdm.notebook import trange
from collections import OrderedDict

#create the loop
try:
    with trange(1, n_epochs + 1, desc="All epochs") as epochs:
        for epoch in epochs:
            with trange(1, n_steps + 1, desc="Epoch {}/{}".format(epochs, n_epochs)) as steps:
                for step in steps:
                    X_batch, y_batch = random_batch(X_train, y_train)
                    with tf.GradientTape() as tape:
                        y_pred = model(X_batch)
                        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
                        loss = tf.add_n([main_loss] + model.losses)
                        gradients = tape.gradient(loss, model.trainable_variables)
                        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                        for variable in model.variables:
                            if variable.constraint is not None:
                                variable.assign(variable.constraint(variable))
                        status = OrderedDict()
                        mean_loss(loss)
                        status["loss"] = mean_loss.result().numpy()
                        for metric in metrics:                   
                            metric(y_batch, y_pred)
                            status[metric.name] =  metric.result().numpy()
                            steps.set_postfix(status)
                    y_pred = model(X_valid)
                    status["val_loss"] = np.mean(loss_fn(y_valid, y_pred))
                    status["val_accuracy"] = np.mean(keras.metrics.sparse_categorical_accuracy(tf.constant(y_valid, dtype=np.float32), y_pred))
                    steps.set_postfix(status)
                    for metric in [mean_loss] + metrics:
                        metric.reset_states()
except ImportError as ex:
    print("To run this cell, please install tqdm, ipywidgets and restart Jupyter")